# reaction-network (Demo Notebook): Enumerators

### Author: Matthew McDermott
Last Updated: 12/07/21

**If you use this code in your work, please consider citing the following paper:**

McDermott, M. J., Dwaraknath, S. S., and Persson, K. A. (2021). A graph-based network for predicting chemical reaction pathways in solid-state materials synthesis. 
Nature Communications, 12(1). https://doi.org/10.1038/s41467-021-23339-x

### Imports

In [14]:
import logging
from pymatgen.core.composition import Composition, Element
from pymatgen.ext.matproj import MPRester
from rxn_network.enumerators.basic import BasicEnumerator, BasicOpenEnumerator
from rxn_network.enumerators.minimize import MinimizeGibbsEnumerator, MinimizeGrandPotentialEnumerator
from rxn_network.fireworks import EnumeratorFW
from rxn_network.costs.softplus import Softplus
from rxn_network.entries.entry_set import GibbsEntrySet

from fireworks import Workflow, LaunchPad

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Downloading and modifying entries

First, we acquire entries for phases in the Y-Mn-O chemical system from the Materials Project (MP), a computed materials database containing calculations for over 130,000 materials.

In [15]:
with MPRester() as mpr:  # insert your Materials Project API key here if it's not stored in .pmgrc.yaml
    entries = mpr.get_entries_in_chemsys("Y-Mn-O", inc_structure="final")

The `GibbsEntrySet` class allows us to automatically convered `ComputedStructureEntry` objects downloaded from the MP database into `GibbsComputedEntry` objects, where DFT-calculated energies have been converted to an AI-estimated equivalent values of the Gibbs free energies of formation, $\Delta G_f$ for all entries at the specified temperature.

For more information, check out the citation in the documentation for `GibbsComputedEntry`.

In [16]:
temp = 800  # units: Kelvin
gibbs_entries = GibbsEntrySet.from_entries(entries, temp)

We can print the entries by calling `.entries` or `.entries_list`:

In [17]:
gibbs_entries.entries

{GibbsComputedEntry | mp-1062072 | Y1 Mn2 (YMn2)
 Gibbs Energy (800 K) = 2.2154,
 GibbsComputedEntry | mp-1086672 | Mn2 O6 (MnO3)
 Gibbs Energy (800 K) = -2.6940,
 GibbsComputedEntry | mp-1105767 | Mn6 O12 (MnO2)
 Gibbs Energy (800 K) = -23.9704,
 GibbsComputedEntry | mp-1172875 | Mn32 O48 (Mn2O3)
 Gibbs Energy (800 K) = -125.2360,
 GibbsComputedEntry | mp-1178684 | Y2 O4 (YO2)
 Gibbs Energy (800 K) = -15.4520,
 GibbsComputedEntry | mp-1180876 | Mn12 O16 (Mn3O4)
 Gibbs Energy (800 K) = -41.8356,
 GibbsComputedEntry | mp-1187739 | Y3 (Y)
 Gibbs Energy (800 K) = 0.0000,
 GibbsComputedEntry | mp-1187855 | Y2 O2 (YO)
 Gibbs Energy (800 K) = -10.8508,
 GibbsComputedEntry | mp-1189335 | Y4 O12 (YO3)
 Gibbs Energy (800 K) = -23.7903,
 GibbsComputedEntry | mp-1189857 | Mn6 O12 (MnO2)
 Gibbs Energy (800 K) = -24.3435,
 GibbsComputedEntry | mp-1203190 | Mn32 O48 (Mn2O3)
 Gibbs Energy (800 K) = -78.7365,
 GibbsComputedEntry | mp-1204718 | Mn32 O48 (Mn2O3)
 Gibbs Energy (800 K) = -117.8349,
 Gibbs

The `GibbsEntrySet` class has many helpful functions, such as the following `filter_by_stability()` function, which automatically removes entries which are a specified energy per atom above the convex hull of stability:

In [18]:
filtered_entries = gibbs_entries.filter_by_stability(0.025)

You should now see a much shorter list of entries within the Y-Mn-O system (< 25 meV/atom below the hull)

In [19]:
filtered_entries.entries_list

[GibbsComputedEntry | mp-35 | Mn29 (Mn)
 Gibbs Energy (800 K) = 0.0000,
 GibbsComputedEntry | mp-1172875 | Mn32 O48 (Mn2O3)
 Gibbs Energy (800 K) = -125.2360,
 GibbsComputedEntry | mp-18759 | Mn6 O8 (Mn3O4)
 Gibbs Energy (800 K) = -22.7493,
 GibbsComputedEntry | mp-18922 | Mn5 O8 (Mn5O8)
 Gibbs Energy (800 K) = -19.4691,
 GibbsComputedEntry | mp-999539 | Mn4 O4 (MnO)
 Gibbs Energy (800 K) = -12.8711,
 GibbsComputedEntry | mp-1279979 | Mn1 O2 (MnO2)
 Gibbs Energy (800 K) = -4.0968,
 GibbsComputedEntry | mp-12957 | O8 (O2)
 Gibbs Energy (800 K) = 0.0000,
 GibbsComputedEntry | mp-1187739 | Y3 (Y)
 Gibbs Energy (800 K) = 0.0000,
 GibbsComputedEntry | mp-18831 | Y4 Mn4 O14 (Y2Mn2O7)
 Gibbs Energy (800 K) = -53.2849,
 GibbsComputedEntry | mp-2652 | Y16 O24 (Y2O3)
 Gibbs Energy (800 K) = -141.2242,
 GibbsComputedEntry | mp-22508 | Y1 Mn12 (YMn12)
 Gibbs Energy (800 K) = -0.1268,
 GibbsComputedEntry | mp-510598 | Y4 Mn8 O20 (YMn2O5)
 Gibbs Energy (800 K) = -70.2802,
 GibbsComputedEntry | mp-19

## Running enumerators

There are several types of enumerator classes contained within `rxn_network.enumerators`: These are:

1. `BasicEnumerator`: use a combinatorial approach to identify all possible (closed) reactions within a set of entries
2. `BasicOpenEnumerator`: use a combinatorial approach to identify all **open** reactions within a set of entries and a list of specified open entries/elements
3. `MinimizeGibbsEnumerator`: use a thermodynamic approach to identify all reactions within a set of entries that are predicted by minimizing Gibbs free energy between a set of two reacting phases touching at an interface
4. `MinimizeGrandPotentialEnumerator`: use a thermodynamic approach to identify all reactions within a set of entries that are predicted by minimizing the grand potential between a set of two reacting phases touching at an interface with an **open** element at a specified chemical potential

### Basic enumerators
We first create a basic enumerator object by initializing one from the `BasicEnumerator` class:

In [20]:
be = BasicEnumerator()

The `BasicEnumerator` class, as is true for all other enumerator classes, can be provided with several arguments for customizing the enumerator output:

- **precursors**: Optional list of precursor formulas; only reactions which contain at least these phases as reactants will be enumerated.
- **target**: Optional formula of target; only reactions which include formation of this target will be enumerated.
- **calculators**: Optional list of Calculator object names; see calculators module for options (e.g., ["ChempotDistanceCalculator])
- **n**: Maximum reactant/product cardinality; i.e., largest possible number of entries on either side of the reaction. Defaults to 2.
- **exclusive_precursors**: Whether to consider only reactions that have reactants which are a subset of the provided list of precursors. Defaults to True.
- **exclusive_targets**: Whether to consider only reactions that make the provided target directly (i.e. with no byproducts). Defualts to False.
- **remove_unbalanced**: Whether to remove reactions which are unbalanced. Defaults to True.
- **remove_changed**: Whether to remove reactions which can only be balanced by removing a reactant/product or having it change sides. Defaults to True.

Note that the default arguments are good for generating a list of simple (unconstrained) reactions, as we might build for a reaction network. Run the following cell:

In [21]:
all_rxns = be.enumerate(filtered_entries)

BasicEnumerator:   0%|          | 0/4 [00:00<?, ?it/s]

This should complete somewhat quickly (within a few seconds). As a result, a list of ~800 generated reactions will be stored within the `all_rxns` object:

In [23]:
print(len(all_rxns))

800


In [24]:
all_rxns

[Mn5O8 + 2 YMn2O5 -> 9 MnO2 + 2 Y,
 MnO + 0.01449 Y2O3 -> 0.1304 Mn5O8 + 0.02899 YMn12,
 2 YMnO3 + 0.3333 Y2O3 -> Y2Mn2O7 + 0.6667 Y,
 0.1429 Y2Mn2O7 + 0.05952 YMn12 -> MnO + 0.3452 Y,
 YMnO3 + 3 MnO -> Y + 2 Mn2O3,
 Y2Mn2O7 + 0.2857 Mn -> 0.4286 Y2O3 + 1.143 YMn2O5,
 2 YMn12 + 14.5 O2 -> Y2Mn2O7 + 22 MnO,
 Mn3O4 + 1.472 Y -> 1.333 YMnO3 + 0.1389 YMn12,
 MnO + YMn2O5 -> YMnO3 + Mn2O3,
 4 MnO + Y2O3 -> Y2Mn2O7 + 2 Mn,
 2.029 Y2Mn2O7 + 0.07843 YMn12 -> Mn5O8 + 2.069 Y2O3,
 1.667 MnO + 0.3333 Y -> Mn + 0.3333 YMn2O5,
 Mn3O4 + 2.667 Y -> 3 Mn + 1.333 Y2O3,
 0.07778 YMn12 + 0.03333 YMn2O5 -> Mn + 0.05556 Y2O3,
 0.3333 Mn3O4 -> MnO + 0.1667 O2,
 Y + 6 Mn2O3 -> YMn12 + 9 O2,
 MnO2 + 0.0177 MnO -> 0.2035 Mn5O8 + 0.1947 O2,
 MnO2 + 0.08333 Y -> 0.08333 YMn12 + O2,
 2 Mn3O4 + Y2O3 -> Y2Mn2O7 + 4 MnO,
 Mn + 2 MnO2 -> MnO + Mn2O3,
 Mn3O4 + 2 MnO2 -> Mn5O8,
 0.06667 YMn12 + 0.9333 YMn2O5 -> YMnO3 + 1.667 MnO,
 0.25 Y + 0.75 YMn2O5 -> YMnO3 + 0.25 Mn2O3,
 MnO2 + 0.4444 Y -> 0.3333 Mn3O4 + 0.2222 Y2O

Looking at the list of reactions above, we see that all reactions are stoichiometrically balanced. If we look at any particular reaction object, we find that the reaction energy and uncertainty can automatically be calculated using the entry energies:

In [30]:
r = all_rxns[0]
print(r)
print(r.energy_per_atom, "±", r.energy_uncertainty_per_atom, "eV/atom")

Mn5O8 + 2 YMn2O5 -> 9 MnO2 + 2 Y
0.6116424704410034 ± 0.058671377679908814 eV/atom


If we want to generate only reactions which contain specific precursors, we can supply them when we initialize the `BasicEnumerator` object.

In [37]:
be_precursors = BasicEnumerator(precursors=["Y2O3"])
y2o3_rxns_exclusive = be_precursors.enumerate(filtered_entries)
y2o3_rxns_exclusive

BasicEnumerator:   0%|          | 0/1 [00:00<?, ?it/s]

[0.5 Y2O3 -> Y + 0.75 O2]

Note that by default, this only produces reactions which exclusively have the provided precursor(s). To include reactions that contain this precursor (non-exclusively), set the `exclusive_precursors=False`:

In [38]:
be_precursors = BasicEnumerator(precursors=["Y2O3"], exclusive_precursors=False)
y2o3_rxns = be_precursors.enumerate(filtered_entries)
y2o3_rxns

BasicEnumerator:   0%|          | 0/4 [00:00<?, ?it/s]

[MnO2 + 0.5 Y2O3 -> 0.5 Y2Mn2O7,
 0.4 Mn5O8 + 1.267 Y2O3 -> Y2Mn2O7 + 0.5333 Y,
 MnO + 0.01449 Y2O3 -> 0.1304 Mn5O8 + 0.02899 YMn12,
 Mn + 0.06863 Y2O3 -> 0.02941 Y2Mn2O7 + 0.07843 YMn12,
 Y2O3 + 207 Mn2O3 -> 78 Mn5O8 + 2 YMn12,
 2 YMnO3 + 0.3333 Y2O3 -> Y2Mn2O7 + 0.6667 Y,
 4 MnO + Y2O3 -> Y2Mn2O7 + 2 Mn,
 0.4286 Y2O3 + 1.143 YMn2O5 -> Y2Mn2O7 + 0.2857 Mn,
 0.1667 Y2O3 + 0.5 Mn2O3 -> MnO2 + 0.3333 Y,
 2 Mn3O4 + Y2O3 -> Y2Mn2O7 + 4 MnO,
 2 Y2O3 + 3 Mn2O3 -> Y + 3 YMn2O5,
 0.08333 YMn12 + 0.6667 Y2O3 -> MnO2 + 1.417 Y,
 Mn + 0.03922 Y2O3 -> 0.05882 MnO2 + 0.07843 YMn12,
 5 Mn + 2.667 Y2O3 -> Mn5O8 + 5.333 Y,
 0.4167 YMn12 + 2.667 Y2O3 -> Mn5O8 + 5.75 Y,
 Mn5O8 + Y2O3 -> MnO + 2 YMn2O5,
 Mn + 0.07692 Y2O3 -> 0.07692 YMnO3 + 0.07692 YMn12,
 0.3333 Mn3O4 + 0.2222 Y2O3 -> MnO2 + 0.4444 Y,
 MnO + 0.6667 Y2O3 -> YMnO3 + 0.3333 Y,
 0.6667 Mn3O4 + 1.444 Y2O3 -> Y2Mn2O7 + 0.8889 Y,
 Y2O3 + 2 YMn2O5 -> Y2Mn2O7 + 2 YMnO3,
 3 Mn + 1.333 Y2O3 -> Mn3O4 + 2.667 Y,
 Mn + 0.0381 Y2O3 -> 0.02857 Mn3O4 + 

This same approach can be used for the target phases as well:

In [42]:
be_target = BasicEnumerator(targets=["YMnO3"])
ymno3_rxns = be_target.enumerate(filtered_entries)
ymno3_rxns

BasicEnumerator:   0%|          | 0/4 [00:00<?, ?it/s]

[3 MnO + YMn12 -> YMnO3 + 14 Mn,
 39 MnO2 + YMn12 -> YMnO3 + 25 Mn2O3,
 Mn3O4 + 1.472 Y -> 1.333 YMnO3 + 0.1389 YMn12,
 MnO2 + 0.4 Y -> 0.2 Mn3O4 + 0.4 YMnO3,
 MnO + YMn2O5 -> YMnO3 + Mn2O3,
 YMn2O5 -> YMnO3 + MnO2,
 Mn3O4 + YMn2O5 -> YMnO3 + 2 Mn2O3,
 MnO + 0.3889 Y -> 0.3333 YMnO3 + 0.05556 YMn12,
 0.5 Y2Mn2O7 + 0.5 Mn2O3 -> YMnO3 + MnO2,
 Mn + 2 YMn2O5 -> Mn3O4 + 2 YMnO3,
 Mn + YMn2O5 -> YMnO3 + 2 MnO,
 0.06667 YMn12 + 0.9333 YMn2O5 -> YMnO3 + 1.667 MnO,
 Y + 5 Mn2O3 -> 3 Mn3O4 + YMnO3,
 0.25 Y + 0.75 YMn2O5 -> YMnO3 + 0.25 Mn2O3,
 0.01905 YMn12 + 0.981 YMn2O5 -> YMnO3 + 0.2381 Mn5O8,
 Y + 1.44 YMn2O5 -> 2.4 YMnO3 + 0.04 YMn12,
 YMn12 + 9.75 O2 -> YMnO3 + 5.5 Mn2O3,
 0.4875 Y2Mn2O7 + 0.025 YMn12 -> YMnO3 + 0.1375 Mn2O3,
 0.2727 YMn12 + 2.409 O2 -> Mn3O4 + 0.2727 YMnO3,
 0.625 Y2Mn2O7 + 0.025 YMn12 -> YMnO3 + 0.275 YMn2O5,
 Mn + 0.07692 Y2O3 -> 0.07692 YMnO3 + 0.07692 YMn12,
 0.375 Mn5O8 + Y -> YMnO3 + 0.875 Mn,
 0.5 Y2Mn2O7 + 0.625 Mn3O4 -> YMnO3 + 0.375 Mn5O8,
 0.5 Y2Mn2O7 + Mn3O4 

In [44]:
be_target = BasicEnumerator(targets=["YMnO3"], exclusive_targets=True)
ymno3_rxns = be_target.enumerate(filtered_entries)
ymno3_rxns

BasicEnumerator:   0%|          | 0/1 [00:00<?, ?it/s]

[Y2O3 + Mn2O3 -> 2 YMnO3]

In [13]:
be_target = BasicOpenEnumerator(["O2"],targets=["YMnO3"])
ymno3_rxns = be_target.enumerate(filtered_entries)

BasicOpenEnumerator:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
ymno3_rxns

[3 Y2Mn2O7 + MnO -> 6 YMnO3 + MnO2 + O2,
 0.3333 Mn3O4 + YMn2O5 -> YMnO3 + 0.1667 O2 + Mn2O3,
 YMn12 + 10.6 O2 + MnO -> YMnO3 + 2.4 Mn5O8,
 O2 + Y + MnO -> YMnO3,
 43 Mn5O8 + YMn12 -> YMnO3 + O2 + 113 Mn2O3,
 Y2Mn2O7 + O2 + 15 Mn2O3 -> 2 YMnO3 + 6 Mn5O8,
 Y2Mn2O7 + YMn12 + O2 -> 3 YMnO3 + 11 Mn,
 0.4 Mn5O8 + YMn2O5 -> Mn3O4 + YMnO3 + 0.6 O2,
 0.5 Y2Mn2O7 -> YMnO3 + 0.25 O2,
 MnO2 + YMn2O5 -> YMnO3 + O2 + 2 MnO,
 0.5 Y2Mn2O7 + 0.3333 Mn3O4 -> YMnO3 + Mn + 0.9167 O2,
 Mn3O4 + 1.5 O2 + Y -> YMnO3 + 2 MnO2,
 0.5 Y2O3 + Mn2O3 -> YMnO3 + Mn + 0.75 O2,
 Mn5O8 + 1.5 O2 + Y -> YMnO3 + 4 MnO2,
 4.273 Y2Mn2O7 + 0.09091 YMn12 -> 8.636 YMnO3 + MnO2 + O2,
 3 Y2Mn2O7 + MnO -> 5 YMnO3 + O2 + YMn2O5,
 0.007874 YMn12 + 0.9921 YMn2O5 -> YMnO3 + 0.1713 O2 + 0.5394 Mn2O3,
 4.333 Mn3O4 + Y2O3 -> YMnO3 + YMn12 + 8.667 O2,
 3 MnO2 + 1.5 Y2O3 -> Y2Mn2O7 + YMnO3 + 0.25 O2,
 0.0177 MnO + YMn2O5 -> YMnO3 + 0.2035 Mn5O8 + 0.1947 O2,
 Y2Mn2O7 + 2.2 Mn5O8 -> YMnO3 + YMn12 + 10.8 O2,
 1.667 Mn + O2 + Y2O3 -> 1.667 YM

#### Open entries

All of the lessons learned above also apply to the `BasicOpenEnumerator` class, although now a list of open entry formulas must be specified.

### Minimize Enumerators

## Launching enumerators with the Fireworks package

In [4]:
fw = EnumeratorFW([be], chemsys=["Y","Mn","O"])

In [7]:
wf = Workflow([fw], name="Y-Mn-O Enumerator")

In [8]:
lpad = LaunchPad.auto_load()
lpad.add_wf(wf)

KeyboardInterrupt: 

In [15]:
fw.tasks[0].run_task(fw.spec)

KeyError: '_fw_env'